In [1]:
import os
from pathlib import Path

#auto load the eemeter module
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import numpy as np

from hourly_test_utils import *
from applied_data_science.bigquery.data import Meter_Data
from eemeter import eemeter as em
from eemeter.common.metrics import BaselineTestingMetrics as Metrics

import multiprocessing as mp

In [2]:
dataset = 'mce_3_yr_precovid'
cache_dir = Path("/app/.recurve_cache/data").resolve()

if 'data' in globals():
    del data
for subsample in range(1, 7):
    for has_solar in [True, False]:
        data = Meter_Data(dataset, subsample, "hourly", solar=has_solar, cache_dir=cache_dir)
        meta = data.df['meta']
        subsample_df = data.df['meter']
        ids = subsample_df.index.unique()

        settings = em.HourlySettings(
            TRAIN_FEATURES=['ghi'],
            LAGGED_FEATURES=None, # 'ghi'
            WINDOW=None,
            OUTPUT_FEATURES=['start_local', 'temperature', 'ghi', 'clearsky_ghi', 'observed', 'predicted', 'month'],
            SUPPLEMENTAL_DATA=False,
            ALPHA=0.1,
            L1_RATIO=0.1,
            SELECTION="cyclic",
            MAX_ITER=1000,
            SEED=42
        )

        arglist = []
        for sid in ids:
            lat = meta.loc[meta.index == sid].iloc[0]["station_latitude"]
            lon = meta.loc[meta.index == sid].iloc[0]["station_longitude"]
            sd = subsample_df.loc[sid].copy()
            metadata = {"lat": lat, "lon": lon, "sid": sid}
            arglist.append([sid, has_solar, subsample, sd, metadata, settings])

        with mp.Pool(processes=mp.cpu_count() - 1) as pool:
            pool.map(save_features, arglist)
        
        print(f"Finished subsample {subsample} with solar {has_solar}")
   

Loading: /app/.recurve_cache/data/MCE/MCE_3_yr_precovid_solar_meta_1.pkl
Done in 0.007 s
Loading: /app/.recurve_cache/data/MCE/MCE_3_yr_precovid_solar_hourly_meter_1.pkl
Done in 8.769 s
Created path: /app/.recurve_cache/mce_3_yr_precovid/MCE_features/True/1
Finished subsample 1 with solar True
Loading: /app/.recurve_cache/data/MCE/MCE_3_yr_precovid_non_solar_meta_1.pkl
Done in 0.008 s
Loading: /app/.recurve_cache/data/MCE/MCE_3_yr_precovid_non_solar_hourly_meter_1.pkl
Done in 5.175 s
Created path: /app/.recurve_cache/mce_3_yr_precovid/MCE_features/False/1
Finished subsample 1 with solar False
Loading: /app/.recurve_cache/data/MCE/MCE_3_yr_precovid_solar_meta_2.pkl
Done in 0.009 s
Loading: /app/.recurve_cache/data/MCE/MCE_3_yr_precovid_solar_hourly_meter_2.pkl
Done in 8.354 s
Created path: /app/.recurve_cache/mce_3_yr_precovid/MCE_features/True/2
Finished subsample 2 with solar True
Loading: /app/.recurve_cache/data/MCE/MCE_3_yr_precovid_non_solar_meta_2.pkl
Done in 0.006 s
Loading: /ap

In [3]:
#check the size of MCE_features folder
!du -h /app/.recurve_cache/data/mce_3_yr_precovid/hourly/subsample_1/MCE_features

E0326 19:39:07.419160598 2205850 backup_poller.cc:127]                 Run client channel backup poller: UNKNOWN:pollset_work {created_time:"2024-03-26T19:39:07.415018862+00:00", children:[UNKNOWN:Bad file descriptor {syscall:"epoll_wait", os_error:"Bad file descriptor", errno:9, created_time:"2024-03-26T19:39:07.414918325+00:00"}]}
du: cannot access '/app/.recurve_cache/data/mce_3_yr_precovid/hourly/subsample_1/MCE_features': No such file or directory
